In [57]:
import os 
import pandas as pd
from django.apps import apps as django_apps
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

## Data Fix for PID B142-040990022-1 & B142-040990022-1-60

Remove schedule with 2000M & 2000, including the CRFs and PRNs (Enrollment schedules)

**NB: Leave other schedules**


In [58]:
from edc_appointment.models import Appointment as CaregiverAppointment
from flourish_child.models import Appointment as ChildAppointment
from django.core.exceptions import ObjectDoesNotExist

caregiver_subject_identifier = 'B142-040990022-1'
child_subject_identifier = 'B142-040990022-1-60'

### CAREGIVER DATA FIX

In [59]:

try:
    onschedule_instance = OnScheduleCohortAEnrollment.objects.get(
        subject_identifier = caregiver_subject_identifier,
        child_subject_identifier = child_subject_identifier)

    subject_history_instance = SubjectScheduleHistory.objects.get(
        subject_identifier = caregiver_subject_identifier,
        onschedule_model = 'flourish_caregiver.onschedulecohortaenrollment')

    appointment = CaregiverAppointment.objects.get(
        subject_identifier = caregiver_subject_identifier,
        visit_code = '2000M',
        schedule_name = 'a_enrol2_schedule1',
        visit_code_sequence = '0')

except ObjectDoesNotExist:
    pass
else:
    appointment.delete()
    onschedule_instance.delete()
    subject_history_instance.delete()

### CHILD DATA FIX

In [60]:
child_onschedule_instance = OnScheduleChildCohortAEnrollment.objects.filter(
    subject_identifier = child_subject_identifier,)

child_subject_history_instance = SubjectScheduleHistory.objects.filter(
    subject_identifier = child_subject_identifier,
    onschedule_model = 'flourish_child.onschedulechildcohortaenrollment')

try:

    child_appointment = ChildAppointment.objects.get(
        subject_identifier = child_subject_identifier,
        schedule_name = 'child_a_enrol_schedule1',
        visit_code = '2000',
        visit_code_sequence = '0')
    
    child_visit = ChildVisit.objects.get(
        appointment = child_appointment)
    
except (ChildAppointment.DoesNotExist, 
        ChildVisit.DoesNotExist):
    pass
else:

    child_requisitions = ChildRequisition.objects.filter(
     child_visit = child_visit)

    child_requisitions.delete()
    child_visit.delete() 
    child_appointment.delete()
    child_onschedule_instance.delete()
    child_subject_history_instance.delete()